# ブロック崩しを学習する

Stable BaselinesとRL Baselines Zooを用いて、強化学習によるブロック崩しゲームの学習を試します。

CartPole、LunarLanderと同様に、まずStable Baselinesで学習の詳細の流れを確認したあと、RL Baselines Zooのお手軽1行コマンドで学習、結果の可視化を行います。

このノートブックは以下の内容を含みます。

- 環境準備
- Gym環境とエージェントを作成
- エージェントの学習と評価
- 1行のコマンドで学習
- リプレイ動画の生成


なお、GIFアニメーションによる学習前後のプレイの可視化は、ikeyasu氏の[ChainerRL を Colaboratory で動かす - Qiita](https://qiita.com/ikeyasu/items/ec3c88ce13a2d5e41f26) を参考として作成しました。

## A. 環境を準備する

Stable Baselinesと依存ライブラリをインストールします。

### 1. 必要なライブラリのインストール

インストールに5分程度を要します。

In [ ]:
# added on 2020/2/9
%tensorflow_version 1.x
# エラー解消のため最新のstable-baselinesをインストール
!pip install optuna
!git clone https://github.com/hill-a/stable-baselines
%cd stable-baselines
!pip install -e .[docs,tests]

In [0]:
!apt-get -y install swig xvfb python-opengl
!pip install box2d box2d-kengz pybullet pyyaml pytablewriter pyvirtualdisplay

### 2. 必要ライブラリ、方策などをインポート

In [0]:
import os
import numpy as np

from stable_baselines.common.cmd_util import make_atari_env
from stable_baselines.deepq.policies import CnnPolicy
from stable_baselines.common.vec_env import VecFrameStack
from stable_baselines import DQN

from pyvirtualdisplay import Display

import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import HTML

## B. Gym環境を準備する


Atari環境を整備してくれる関数(make_atari_env)があるため、それを使います。

- num_env: 並列する環境数
- VecFrameStack: スタックするフレーム数を指定

In [0]:
env = make_atari_env('BreakoutNoFrameskip-v4', num_env=1, seed=0)

## C. その他の処理を準備する

Gym環境以外のヘルパー関数などを準備します。

### 1. プレイ動画の再生用関数

次に、仮想ディプレイを利用し、Colaboratory上でエージェントの振る舞いをアニメーションで見られるようにする関数を定義します。

In [0]:
def playback(model, env, maxsteps):
  # Start virtual display
  display = Display(visible=0, size=(1024, 768))
  display.start()

  os.environ["DISPLAY"] = ":" + str(display.display) + "." + str(display.screen)

  frames = []
  for i in range(3):
      obs = env.reset()
      done = False
      R = 0
      t = 0
      while not done and t < maxsteps:
          frames.append(env.render(mode = 'rgb_array'))
          action, _states = model.predict(obs)        
          obs, rewards, dones, info = env.step(action)
          R += rewards
          t += 1
      print('test episode:', i, 'R:', R)
  #    model.stop_episode()
  #env.render()

  return frames

## D. モデルを作成する

### 1. エージェントの準備

In [0]:
# policy: 'CnnPolicy'
# n_timesteps: !!float 1e7
# buffer_size: 10000
# learning_rate: !!float 1e-4
# learning_starts: 10000
# target_network_update_freq: 1000
# train_freq: 4
# exploration_final_eps: 0.01
# exploration_fraction: 0.1
# prioritized_replay_alpha: 0.6
# prioritized_replay: True
#n_timesteps=int(1e7), 

model = DQN(CnnPolicy, env, verbose=1, buffer_size=10000, learning_rate=int(1e4), learning_starts=10000, target_network_update_freq=1000, train_freq=4, exploration_final_eps=0.01, exploration_fraction=0.1, prioritized_replay_alpha=0.6, prioritized_replay=True, tensorboard_log="./breakout_tensorboard/")

### 4. 未学習状態での振る舞いを見る

この未学習状態でのプレイぶりを再生してみましょう。

In [0]:
frames = playback(model, env, 100)

plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
patch = plt.imshow(frames[0])
plt.axis('off')
animate = lambda i: patch.set_data(frames[i])
ani = matplotlib.animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval = 50)
HTML(ani.to_jshtml())

## E. エージェントを学習させる

### 1. TensorBoardのインライン起動

TensorBoardをインラインで起動し、学習の進捗をモニタできるようにします。


In [0]:
%load_ext tensorboard
%tensorboard --logdir breakout_tensorboard/BreakoutNoFrameskip-v4

### 2. エージェントを学習させ、プレイバックを確認する

1万ステップの学習をしてみます。

In [0]:
n_timesteps = 10000
model.learn(n_timesteps)

まだ、これでは全くゲームのプレイの体を成していません。

In [0]:
frames = playback(model, env, 100)

plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
patch = plt.imshow(frames[0])
plt.axis('off')
animate = lambda i: patch.set_data(frames[i])
ani = matplotlib.animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval = 50)
HTML(ani.to_jshtml())

### 3. RL Baselines Zooの導入

今回も、ここからは、

- 環境を準備する
- エージェントを作成する
- 環境上で、エージェントに探索させる

というステップを1行のコマンドの裏側にまとめてくれるRL Baselines Zooを使ってみましょう。

例によって、まずRL Baselines Zooのリポジトリをコピーします。


In [0]:
%cd /content
!git clone https://github.com/araffin/rl-baselines-zoo.git
%cd /content/rl-baselines-zoo

### 4. 1行のコマンドで学習する

いよいよエージェントの学習を開始します。今回はDQNというモデルを使います。

DQNで `BreakOutNoFrameskip-v4` 環境での学習を進める時、どのようなハイパーパラメータが事前設定されているか、確認してみましょう。

以下のコマンドで、DQN用のパラメータを確認できます。 `atari:` 以下に書かれた内容を確認しましょう。

In [0]:
!cat /content/rl-baselines-zoo/hyperparams/dqn.yml

以下のようなパラメータが設定されているようです。

```
atari:
  policy: 'CnnPolicy'
  n_timesteps: !!float 1e7
  buffer_size: 10000
  learning_rate: !!float 1e-4
  learning_starts: 10000
  target_network_update_freq: 1000
  train_freq: 4
  exploration_final_eps: 0.01
  exploration_fraction: 0.1
  prioritized_replay_alpha: 0.6
  prioritized_replay: True
```

### 5. 学習の開始

以下のコマンドを実行したら、TensorBoardのセルに戻り、reward、lossの変化の様子を確認してみましょう。

また、実行中にランタイムへの接続が切れた場合は、メニュー右上の「再接続」ボタンより、再度繋ぐことができます。

In [0]:
!python train.py --algo dqn --env BreakoutNoFrameskip-v4 --tensorboard-log breakout_tensorboard/

実行すると、 `logs/dqn/BreakoutNoFrameskip-v4_1/` に  `BreakoutNoFrameskip-v4.zip` として、学習済みエージェントのモデルが保存さます。

## F. 学習済みエージェントの評価

保存された学習済みエージェントのモデルを使って、振る舞いを観察してみましょう。

以下のセルを実行すると、出力セルで、アニメーションを再生できます。これは、今回あなたが学習させたエージェントのプレイする様子です！

In [0]:
frames = playback(model, env, 300)

plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
patch = plt.imshow(frames[0])
plt.axis('off')
animate = lambda i: patch.set_data(frames[i])
ani = matplotlib.animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval = 50)
HTML(ani.to_jshtml())

## まとめと発展

このノートブックでは、Atariゲームのひとつ、BreakOut（ブロック崩し）に強化学習エージェントを習熟させることができました。

コードの一部を改変することで、他のAtariゲームに対する学習を行ったり、学習済みエージェントの再生を行うことができます。試してみましょう。